In [1]:
#%matplotlib widget

%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import glob
from astropy.io import fits
import astropy.units as u
from astropy.visualization import ImageNormalize
from astropy.time import Time
from astropy.visualization import time_support
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patheffects as path_effects
from astropy.visualization import LogStretch, ZScaleInterval, ImageNormalize, PercentileInterval, AsinhStretch, LinearStretch
from sunpy.net import Fido
from sunpy.net import attrs as a
import sunpy.map
from astropy.wcs import WCS, wcs
import os
from sunpy.coordinates import get_body_heliographic_stonyhurst, get_horizons_coord

In [15]:
#Initial settings

#Option 1) download WISPR images directly from VSO
download = 1

#Option 2) if you wanna use your file, uncomment. Then also set files_wispr variable to path_L3file
#download = 0
#path_L3file = '/Users/gretacappello/Desktop/jupyter_notebooks/thomson_scattering/material_andrea/Examples/example_1/example_1_LW/psp_LW_wispr_20181106T154054_V3_1221.fits'
#files_wispr = [path_L3file]

#Set example number. If you wanna track multiple time, you must change the example value from 1 to 3 and run the code 3 times. It will create each time a folder with the examlpe number included in the name.
example = 1
key = "real"
nm = 0

#Insert your directory to save figures
path_figures = '/Users/gretacappello/Desktop/PROJECT_2_METIS_TS/PSI_simulation_erika/plots/' 

#select time for which you wanna do the analysis. Note: to download a single or a few frames, you must restrict the interval of time.
attrs_time = a.Time('2021/04/29 11:57:00', '2021/04/29 12:05:00')


#other keywords
#key = "fake" 
#key = 'raytracing'

#longitude = 175
#latitude = 21 #'M10'

In [581]:
if key == "real" and download == 1:
    #Download WISPR data
    result_wispr = Fido.search(attrs_time, a.Instrument('WISPR'), a.Detector('INNER'), a.Level('3')) 
    #result_wispr = Fido.search(attrs_time, a.Instrument('WISPR'), a.Detector('INNER'), a.Level('2')) 
    print(result_wispr)
    files_wispr= Fido.fetch(result_wispr[0])  #file if you wanna download

In [6]:
if key == "real":
    for i in range(len(files_wispr)):
        print(i, files_wispr[i])
    path =files_wispr[0]
    print(path)

    files = glob.glob(path)
    files = sorted(files)

    hea= []
    ima = []

    for i in range(len(files)):
        hea.append([])
        ima.append([])
    for i in range(len(files)):   
        hea[i] = fits.getheader(files[i])
        ima[i] = fits.getdata(files[i])

    header = hea[0]
    images = ima[0]
    #make a maps' sequence

    m_seq = sunpy.map.Map(files, sequence = True)

0 /Users/gretacappello/Desktop/jupyter_notebooks/thomson_scattering/material_andrea/Examples/example_1/example_1_LW/psp_LW_wispr_20181106T154054_V3_1221.fits
/Users/gretacappello/Desktop/jupyter_notebooks/thomson_scattering/material_andrea/Examples/example_1/example_1_LW/psp_LW_wispr_20181106T154054_V3_1221.fits


In [7]:
if key == "fake":

    path = '/Users/gretacappello/Downloads/wispr_i+o_mask_tb000320.fts'

    f1 = glob.glob(path)
    f1 = sorted(f1)
    m_seq = sunpy.map.Map(f1, sequence = True)
    m_seq[0]
    header = fits.getheader(path)
    images  = fits.getdata(path)
    print(Time(header['DATE_OBS']))
    obstime = Time(header['DATE_OBS'])  # Make sure the correct time is assigned here
    #Get the position of Parker Solar Probe at the observation time
    get_horizons_coord("Parker Solar Probe",obstime)
    DSUN_OBS_solar_radii = header['DSUN_OBS']
    print(DSUN_OBS_solar_radii)
    header['R_SUN'] =DSUN_OBS_solar_radii
    header['DATE-OBS'] = header['DATE_OBS']
    

In [8]:
if key == 'raytracing':
    directory_rt = '/Users/gretacappello/Desktop/PROJECT_2_METIS_TS/PSI_simulation_erika/raytracingHCS_29March2024/' 
    path = directory_rt +'raytracingHCS_lon'+str(longitude) + 'lat'+str(latitude) + '/synth_ch_psp_l3_wispr_20240329t180015_v1_1211.fits'
    f_rt = glob.glob(path)
    f_rt = sorted(f_rt)
    m_seq = sunpy.map.Map(f_rt, sequence = True)
    m_seq[0]
    header = fits.getheader(path)
    images  = fits.getdata(path)
    print(Time(header['DATE-OBS']))
    obstime = Time(header['DATE-OBS'])  # Make sure the correct time is assigned here
    #Get the position of Parker Solar Probe at the observation time
    #get_horizons_coord("Parker Solar Probe",obstime)
    DSUN_OBS_solar_radii = header['DSUN_OBS']
    print(DSUN_OBS_solar_radii)
    header['R_SUN'] = DSUN_OBS_solar_radii
    header['DATE-OBS'] = header['DATE-OBS']
    #print(header)

In [9]:
#Create a folder with the example number
print(example)
folderr = os.path.join(path_figures, f"test_{example}")
if not os.path.exists(folderr):
    os.makedirs(folderr)
cartella = folderr + "/"

1


In [11]:
#Apply some filters to WISPR images

def meters_to_solar_radii(distance_meters):
    solar_radius_meters = 6.957e8  # Solar radius in meters
    return distance_meters / solar_radius_meters



# Create WCS object
wcs = WCS(header)
print(wcs)

# Extract pixel grid 
ny, nx = images.shape  # Get the image dimensions (ny: rows, nx: columns)
x = np.arange(nx)
y = np.arange(ny)
x_grid, y_grid = np.meshgrid(x, y)

# Convert the pixel grid to helioprojective coordinates
helioprojective_coords = wcs.pixel_to_world(x_grid, y_grid)

# Extract HPC longitude (x) and latitude (y) in degree in this case
hpc_x_deg = helioprojective_coords.Tx.degree  # .arcsec for Helioprojective longitude in arcseconds
hpc_y_deg = helioprojective_coords.Ty.degree  #  .arcsec for Helioprojective latitude in arcseconds

# Conversion from deg to radians
hpc_x=np.deg2rad(hpc_x_deg)
hpc_y=np.deg2rad(hpc_y_deg)

# Sanity check :D
print("---------------------")
print("Sanity check :D")
print(f"HPC Longitude: {hpc_x}, HPC Latitude: {hpc_y}")
print("type(hpc_x): ",type(hpc_x))
print("len(hpc_x): ",len(hpc_x))

#RADIAL FILTER by Erika
rfilter = hpc_x**2 + hpc_y**2
print(rfilter)

#dfilter = np.ones(len(hdr1))
for i in range(len(header)):
    if key == "fake":
        psp_r = header['R_SUN']
    if key == "real":
        psp_r = header['DSUN_OBS']
    if key =="raytracing":
        psp_r = header['R_SUN']

    dfilter = (psp_r/np.nanmin(psp_r))**3
    
print(dfilter)



r_filter_map = m_seq[nm] * rfilter * dfilter* rfilter * dfilter #[nm]


Set MJD-BEG to 58428.653402 from DATE-BEG.
Set MJD-AVG to 58428.654256 from DATE-AVG.
Set MJD-END to 58428.655111 from DATE-END'. [astropy.wcs.wcs]


WCS Keywords

Number of WCS axes: 2
CTYPE : 'HPLN-ZPN'  'HPLT-ZPN'  
CRVAL : 32.2994179668  -5.32795557231  
CRPIX : 495.773  507.555  
PC1_1 PC1_2  : 0.985859296508  0.142633434984  
PC2_1 PC2_2  : -0.142494542251  0.985575239292  
CDELT : 0.042305  0.042305  
NAXIS : 960  1024
---------------------
Sanity check :D
HPC Longitude: [[0.17318639 0.17377015 0.17435441 ... 0.84169937 0.84235796 0.84301613]
 [0.17326626 0.17385012 0.17443448 ... 0.84180386 0.84246246 0.84312062]
 [0.17334624 0.17393021 0.17451468 ... 0.84190826 0.84256686 0.84322503]
 ...
 [0.28622373 0.28682519 0.28742704 ... 0.91624028 0.91682101 0.91740131]
 [0.28635508 0.28695647 0.28755827 ... 0.91627823 0.91685881 0.91743897]
 [0.28648648 0.28708781 0.28768954 ... 0.91631607 0.9168965  0.91747652]], HPC Latitude: [[-0.36223055 -0.3624215  -0.36261238 ... -0.46032122 -0.46030423
  -0.46028698]
 [-0.36166002 -0.36185084 -0.36204157 ... -0.45974844 -0.4597316
  -0.4597145 ]
 [-0.36108912 -0.36127979 -0.36147039 ... -0.45

See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html forthe FITS unit standards. [sunpy.map.mapbase]


In [12]:
#function to click on feature and determine x and y coordinates in pixels

def onclick(event):
    global x_coords, y_coords
    x_coords.append(event.xdata)
    y_coords.append(event.ydata)
    
    ax.plot(event.xdata, event.ydata, marker = 'x', ls = '', color = 'r', markersize = 4)
    #print(event.xdata, event.ydata)

In [13]:
#x and y coordinates in pixels determined by clicking on L3 data

x_coords = []
y_coords = []

In [14]:
#Click along the ray for each ray to get the coordinates.


fig = plt.figure()
ax = fig.add_subplot(projection=m_seq[nm])
#m_seq[nm].plot(axes=ax, clip_interval=(2, 97)*u.percent)

if key == 'real':
    #filtered map
    #r_filter_map.plot(axes=ax, clip_interval=(2, 97)*u.percent)

    #pure map
    m_seq[0].plot(axes=ax, clip_interval=(1, 99)*u.percent)
if key == 'fake':
    norm = ImageNormalize(r_filter_map.data,
                  interval=PercentileInterval(99.6),  # puoi usare anche ZScaleInterval()
                  stretch=AsinhStretch(0.005))         # o LogStretch()

    cmap = matplotlib.colormaps['binary_r']
    r_filter_map.plot(axes=ax, norm=norm,  cmap=cmap)
if key == 'raytracing':
    #m_seq[0].plot(axes=ax, clip_interval=(2, 93)*u.percent)
    m_seq[0].plot(axes=ax, clip_interval=(1, 99)*u.percent)
print("CLICK:")
cid = fig.canvas.mpl_connect('button_press_event', onclick)

CLICK:


In [591]:
#save coordinates to text file

xx = np.round(np.array(x_coords))
yy = np.round(np.array(y_coords))

coords = np.column_stack([xx,yy])
file_path = os.path.join(cartella, f'points_L3_{example}_points.txt')
np.savetxt(file_path, coords, fmt=['%d', '%d'])


In [592]:
#Fit the tracked point with a Quadratic function.

x = xx
y = yy

# Fit a second-order polynomial (quadratic fit)
coeffs = np.polyfit(x, y, 2)  # Returns [a, b, c] for ax^2 + bx + c
poly_func = np.poly1d(coeffs)  # Create a polynomial function

# Generate 100 evenly spaced x values for interpolation
x_interp = np.linspace(min(x), max(x), 100)
y_interp = poly_func(x_interp)

# Save interpolated data to a new file
x_interp = np.round(np.array(x_interp))
y_interp = np.round(np.array(y_interp))

coords = np.column_stack([x_interp,y_interp])
file_path = os.path.join(cartella, f'interpolated_points_L3_{example}_line.txt')
np.savetxt(file_path, coords, fmt=['%d', '%d'])

fig = plt.figure()
ax = fig.add_subplot(projection=m_seq[nm])
if key == 'real':
    #r_filter_map.plot(axes=ax, clip_interval=(2, 97)*u.percent)
    m_seq[0].plot(axes=ax, clip_interval=(1, 99)*u.percent)
if key == 'fake':
    r_filter_map.plot(axes=ax, norm=norm,  cmap=cmap)
if key == 'raytracing':
   m_seq[0].plot(axes=ax, clip_interval=(1, 99)*u.percent)
# Plot the original data and the fitted curve
plt.scatter(x, y, color='red', label="Tracked points")
plt.plot(x_interp, y_interp, label="Quadratic Fit", color='blue')
#plt.xlabel("X values")
#plt.ylabel("Y values")
plt.legend()
#plt.title("Quadratic Fit of Data")
plt.show()

print(f"Fitted curve data saved to {file_path}")


Fitted curve data saved to /Users/gretacappello/Desktop/PROJECT_2_METIS_TS/PSI_simulation_erika/plots/test_6/interpolated_points_L3_6_line.txt
